<a href="https://colab.research.google.com/github/Akos1993/Pesel-generator-2.0-in-python/blob/main/PESEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Back end

In [8]:
!pip install faker

import sqlite3
import random
from faker import Faker
from datetime import datetime

# Initialize Faker for Polish and other languages
fake = Faker(["pl_PL", "en_US", "es_ES", "de_DE", "fr_FR"])

# Function to generate a valid PESEL based on DoB and sex
def generate_dummy_pesel(dob, sex):
    year = dob.year
    month = dob.month
    day = dob.day

    # Encode century information in the month
    if 1800 <= year < 1900:
        month += 80
    elif 1900 <= year < 2000:
        month += 0
    elif 2000 <= year < 2100:
        month += 20
    elif 2100 <= year < 2200:
        month += 40
    elif 2200 <= year < 2300:
        month += 60

    # Generate random ZZZ (unique identification number)
    zzz = random.randint(0, 999)
    zzz_str = str(zzz).zfill(3)

    # Determine sex digit (X)
    if sex.lower() == "male":
        sex_digit = random.choice([1, 3, 5, 7, 9])
    else:
        sex_digit = random.choice([0, 2, 4, 6, 8])

    # Construct the PESEL without the check digit
    pesel_without_check_digit = f"{year % 100:02}{month:02}{day:02}{zzz_str}{sex_digit}"

    # Calculate the check digit (Q)
    weights = [1, 3, 7, 9, 1, 3, 7, 9, 1, 3, 1]
    check_sum = 0
    for i in range(10):
        check_sum += int(pesel_without_check_digit[i]) * weights[i]

    check_digit = (10 - (check_sum % 10)) % 10

    # Complete the PESEL with the check digit
    pesel = pesel_without_check_digit + str(check_digit)

    return pesel

# Connect to SQLite database (or create it)
conn = sqlite3.connect("dummy_data.db")
cursor = conn.cursor()

# Create table with additional fields for DoB and sex
cursor.execute("""
CREATE TABLE IF NOT EXISTS people (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    surname TEXT,
    dob TEXT,
    sex TEXT,
    pesel TEXT UNIQUE
)
""")

# Generate 10,000 rows of data
data = []
for _ in range(10000):
    first_name = fake.first_name()
    last_name = fake.last_name()

    # Generate a random date of birth
    dob = fake.date_of_birth(minimum_age=18, maximum_age=90)

    # Randomly assign sex
    sex = random.choice(["male", "female"])

    # Generate a PESEL based on the DoB and sex
    pesel = generate_dummy_pesel(dob, sex)

    # Append to data list
    data.append((first_name, last_name, dob.strftime("%Y-%m-%d"), sex, pesel))

# Insert data into the database
cursor.executemany("INSERT INTO people (name, surname, dob, sex, pesel) VALUES (?, ?, ?, ?, ?)", data)

# Commit changes and close the connection
conn.commit()
conn.close()

print("Database created with 10,000 rows!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 58.6 MB/s eta 0:00:00
Database created with 10,000 rows!


In [10]:
class Person:
    def __init__(self, name, surname, sex, dob, pesel_number=None):
        self.name = name
        self.surname = surname
        self.sex = sex
        self.dob = dob
        self.pesel_number = pesel_number


class Pesel:
    def __init__(self, person):
        self.person = person

    def generate(self):
        # Extract date of birth components
        year = self.person.dob.year
        month = self.person.dob.month
        day = self.person.dob.day

        # Encode century information in the month
        if 1800 <= year < 1900:
            month += 80
        elif 1900 <= year < 2000:
            month += 0
        elif 2000 <= year < 2100:
            month += 20
        elif 2100 <= year < 2200:
            month += 40
        elif 2200 <= year < 2300:
            month += 60

        # Generate random ZZZ (unique identification number)
        zzz = random.randint(0, 999)
        zzz_str = str(zzz).zfill(3)

        # Determine sex digit (X)
        if self.person.sex.lower() == 'male':
            sex_digit = random.choice([1, 3, 5, 7, 9])
        else:
            sex_digit = random.choice([0, 2, 4, 6, 8])

        # Construct the PESEL without the check digit
        pesel_without_check_digit = f"{year % 100:02}{month:02}{day:02}{zzz_str}{sex_digit}"

        # Calculate the check digit (Q)
        weights = [1, 3, 7, 9, 1, 3, 7, 9, 1, 3, 1]
        check_sum = 0
        for i in range(10):
            check_sum += int(pesel_without_check_digit[i]) * weights[i]

        check_digit = (10 - (check_sum % 10)) % 10

        # Complete the PESEL with the check digit
        pesel = pesel_without_check_digit + str(check_digit)

        return pesel

    def check_pesel_exists(self, db_path="dummy_data.db"):
        """
        Checks if the generated PESEL already exists in the SQL database.
        """
        try:
            # Connect to the SQLite database
            conn = sqlite3.connect(db_path)

            # Read the 'pesel' column from the 'people' table into a pandas DataFrame
            df = pd.read_sql_query("SELECT pesel FROM people", conn)

            # Close the database connection
            conn.close()

            # Generate a new PESEL number
            pesel = self.generate()

            # Check if the generated PESEL exists in the DataFrame
            return pesel in df['pesel'].values

        except Exception as e:
            print(f"Error checking PESEL in database: {e}")
            return False  # Return False or handle the error appropriately

    def assign_unique_pesel(self):
        pesel = self.generate()
        while self.check_pesel_exists(pesel):
            pesel = self.generate()
        self.person.pesel_number = pesel


In [ ]:
def update_db_with_person(person, db_path="dummy_data.db"):
    """
    Updates the database with the personal data and the successfully generated PESEL.

    Parameters:
        person (Person): The Person object containing the personal data and PESEL.
        db_path (str): The path to the SQLite database file.
    """
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Insert the person's data into the database
        cursor.execute("""
        INSERT INTO people (name, surname, sex, dob, pesel)
        VALUES (?, ?, ?, ?, ?)
        """, (person.name, person.surname, person.sex, person.dob, person.pesel_number))

        # Commit changes and close the connection
        conn.commit()
        conn.close()

        print(f"Successfully added {person.name} {person.surname} to the database with PESEL: {person.pesel_number}")
    except sqlite3.IntegrityError:
        print(f"Error: PESEL {person.pesel_number} already exists in the database.")
    except Exception as e:
        print(f"An error occurred while updating the database: {e}")


In [11]:
# prompt: create an instance of Person and Pesel with 'John' 'Doe' 'male' 1990.05.21.

import pandas as pd

# Create a Person instance
john_doe = Person("John", "Doe", "male", datetime(1990, 5, 21))

# Create a Pesel instance associated with the person
john_doe_pesel = Pesel(john_doe)

# Assign a unique PESEL to the person
john_doe_pesel.assign_unique_pesel()

# Print the person's details including the generated PESEL
print(f"Name: {john_doe.name}")
print(f"Surname: {john_doe.surname}")
print(f"Sex: {john_doe.sex}")
print(f"Date of Birth: {john_doe.dob}")
print(f"PESEL: {john_doe.pesel_number}")

# Update the database with the person's information
update_db_with_person(john_doe)


Error checking PESEL in database: Execution failed on sql 'SELECT pesel FROM people': no such table: people
Name: John
Surname: Doe
Sex: male
Date of Birth: 1990-05-21 00:00:00
PESEL: 90052143051


NameError: name 'update_db_with_person' is not defined

# Front end


In [16]:
!pip install streamlit
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 22 packages in 3s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋

In [23]:
%%writefile app.py

import streamlit as st

class MyApp:
    def __init__(self):
        self.user_input = ""

    def display_title(self):
        st.title("Generowanie Pesel")

    def get_user_input(self):
        self.user_input = st.text_input("")

    def display_output(self):
        if st.button("Potwierdż"):
            st.write(f"Dodana Suksesem {self.user_input}")

# Run the app
if __name__ == "__main__":
    app = MyApp()
    app.display_title()
    app.get_user_input()
    app.display_output()


Overwriting app.py


In [24]:
!streamlit run app.py &>/content/logs.txt &